# Link Prediction Analysis and Evaluation

This notebook evaluates the trained TransE model using comprehensive metrics and generates high-confidence citation predictions. We implement the evaluation pipeline described in the README:

## Evaluation Metrics:
- **Mean Reciprocal Rank (MRR)**: Measures ranking quality
- **Hits@K**: Proportion of correct predictions in top-K
- **AUC & Average Precision**: Binary classification performance

## Analysis Pipeline:
1. Load trained model and test data
2. Comprehensive evaluation with ranking metrics
3. Generate missing citation predictions
4. Qualitative analysis of predictions
5. Performance visualization and reporting

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from dotenv import load_dotenv
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src.db import Neo4jConnection
from src.data_extraction import load_citation_graph
from src.model import TransETrainer
from src.evaluation import LinkPredictionEvaluator, evaluate_model
from src.visualization import (
    plot_evaluation_results,
    plot_prediction_analysis,
    plot_embedding_visualization,
    set_portfolio_style
)

# Set up environment
load_dotenv()
set_portfolio_style()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("Libraries imported successfully!")

## 1. Load Trained Model and Data

In [ ]:
# Load trained model
model_path = "../models/transe_citation_model.pt"
print(f"Loading trained model from {model_path}...")

try:
    trainer = TransETrainer.load_model(model_path, device=device)
    print("✅ Model loaded successfully")
except FileNotFoundError:
    print("❌ Model file not found. Please run 02_model_training.ipynb first.")
    raise

# Load entity mapping
mapping_path = "../models/entity_mapping.pkl"
with open(mapping_path, 'rb') as f:
    entity_mapping = pickle.load(f)

# Load training metadata
metadata_path = "../models/training_metadata.pkl"
with open(metadata_path, 'rb') as f:
    training_metadata = pickle.load(f)

print(f"\n📋 Model Configuration:")
print(f"• Entities: {training_metadata['num_entities']:,}")
print(f"• Embedding dim: {training_metadata['embedding_dim']}")
print(f"• Training epochs: {training_metadata['epochs']}")
print(f"• Final training loss: {training_metadata['final_loss']:.4f}")
print(f"• Model parameters: {sum(p.numel() for p in trainer.model.parameters()):,}")

## 2. Reload Test Data

In [ ]:
# Reconnect to database and reload data for evaluation
print("Reconnecting to database and preparing test data...")

db = Neo4jConnection()
if not db.test_connection():
    raise ConnectionError("Failed to connect to Neo4j database")

# Reload citation graph (needed for consistent train/test split)
extractor = load_citation_graph(db)

# Recreate the same train/test split used during training
training_data = extractor.create_training_data(
    test_size=0.2,
    negative_ratio=1,
    random_state=42  # Same random state as training
)

# Extract test data
test_edges = training_data['test_edges']
test_labels = training_data['test_labels']

# Split into positive and negative test samples
test_pos_mask = test_labels == 1
test_neg_mask = test_labels == 0

test_pos_edges = test_edges[test_pos_mask]
test_neg_edges = test_edges[test_neg_mask]

print(f"\n📊 Test Data:")
print(f"• Total test samples: {len(test_edges):,}")
print(f"• Positive samples: {len(test_pos_edges):,}")
print(f"• Negative samples: {len(test_neg_edges):,}")
print(f"• Test data ready for evaluation")

## 3. Comprehensive Model Evaluation

In [ ]:
# Run comprehensive evaluation
print("Running comprehensive model evaluation...")
print("This may take several minutes for ranking metrics.\n")

# Evaluation parameters
K_VALUES = [1, 3, 10]  # For Hits@K evaluation

# Run evaluation
results = evaluate_model(
    trainer=trainer,
    test_pos_edges=test_pos_edges,
    test_neg_edges=test_neg_edges,
    entity_mapping=entity_mapping,
    k_values=K_VALUES
)

print("\n" + "="*50)
print("📊 EVALUATION RESULTS")
print("="*50)

print(f"\n🎯 Ranking Metrics:")
print(f"• Mean Reciprocal Rank (MRR): {results['mrr']:.4f}")
for k in K_VALUES:
    print(f"• Hits@{k}: {results['hits'][k]:.3f}")

print(f"\n📈 Classification Metrics:")
print(f"• AUC Score: {results['auc']:.4f}")
print(f"• Average Precision: {results['average_precision']:.4f}")

# Store results for visualization
evaluation_results = results

## 4. Performance Interpretation

In [ ]:
# Interpret the results
print("\n🔍 PERFORMANCE INTERPRETATION:")
print("=" * 40)

mrr = results['mrr']
hits_1 = results['hits'][1]
hits_10 = results['hits'][10]
auc = results['auc']

# MRR interpretation
if mrr > 0.3:
    mrr_quality = "Excellent"
elif mrr > 0.2:
    mrr_quality = "Good"
elif mrr > 0.1:
    mrr_quality = "Fair"
else:
    mrr_quality = "Needs improvement"

print(f"\n📊 Ranking Quality: {mrr_quality}")
print(f"   • MRR of {mrr:.4f} means on average, correct citations appear at rank {1/mrr:.1f}")
print(f"   • {hits_1*100:.1f}% of predictions have the correct citation in rank 1")
print(f"   • {hits_10*100:.1f}% of predictions have the correct citation in top 10")

# AUC interpretation
if auc > 0.9:
    auc_quality = "Excellent discrimination"
elif auc > 0.8:
    auc_quality = "Good discrimination"
elif auc > 0.7:
    auc_quality = "Fair discrimination"
else:
    auc_quality = "Poor discrimination"

print(f"\n🎯 Binary Classification: {auc_quality}")
print(f"   • AUC of {auc:.4f} indicates {auc_quality.lower()} between citations and non-citations")
print(f"   • Model can distinguish real from fake citations with {auc*100:.1f}% accuracy")

# Overall assessment
if mrr > 0.15 and auc > 0.75:
    overall = "✅ Strong performance - suitable for citation recommendation"
elif mrr > 0.1 and auc > 0.65:
    overall = "⚠️  Moderate performance - may need hyperparameter tuning"
else:
    overall = "❌ Weak performance - consider model architecture changes"

print(f"\n🏆 Overall Assessment: {overall}")

## 5. Visualize Evaluation Results

In [ ]:
# Create comprehensive evaluation visualization
print("Creating evaluation visualizations...")

fig = plot_evaluation_results(
    evaluation_results,
    figsize=(14, 10),
    save_path="../outputs/evaluation_results.png"
)
plt.show()

print("Evaluation results visualization saved to ../outputs/evaluation_results.png")

## 6. Generate Missing Citation Predictions

In [ ]:
# Generate high-confidence missing citation predictions
print("Generating missing citation predictions...")

# Create evaluator for predictions
evaluator = LinkPredictionEvaluator(
    model=trainer.model,
    entity_mapping=entity_mapping,
    device=device
)

# Get existing citations to exclude from predictions
existing_citations = set()
for source_idx, target_idx in extractor.citation_edges:
    source_paper = extractor.id_to_paper[source_idx]
    target_paper = extractor.id_to_paper[target_idx]
    existing_citations.add((source_paper, target_paper))

print(f"Excluding {len(existing_citations):,} existing citations")

# Sample source papers for prediction (limit for computational efficiency)
all_paper_ids = list(entity_mapping.keys())
np.random.seed(42)
sample_papers = np.random.choice(all_paper_ids, size=min(50, len(all_paper_ids)), replace=False)

print(f"Generating predictions for {len(sample_papers)} sample papers...")

# Generate predictions
predictions_df = evaluator.predict_missing_citations(
    source_papers=sample_papers.tolist(),
    top_k=20,  # Top 20 predictions per paper
    exclude_existing=True,
    existing_citations=existing_citations
)

print(f"\n📋 Generated {len(predictions_df):,} citation predictions")
print(f"• Average predictions per paper: {len(predictions_df) / len(sample_papers):.1f}")
print(f"• Score range: {predictions_df['score'].min():.4f} to {predictions_df['score'].max():.4f}")
print(f"• Lower scores indicate higher likelihood")

## 7. Analyze Top Predictions

In [ ]:
# Examine top predictions
print("\n🏆 TOP 20 CITATION PREDICTIONS:")
print("=" * 80)

top_predictions = predictions_df.head(20)

for idx, row in top_predictions.iterrows():
    source_title = row['source_paper'][:60] + "..." if len(row['source_paper']) > 60 else row['source_paper']
    target_title = row['target_paper'][:60] + "..." if len(row['target_paper']) > 60 else row['target_paper']
    score = row['score']
    rank = row['rank']
    
    print(f"\n{idx+1:2d}. Score: {score:.4f} | Rank: {rank}")
    print(f"    Source: {source_title}")
    print(f"    Target: {target_title}")
    print(f"    {'-'*75}")

# Save predictions to CSV for further analysis
predictions_path = "../outputs/citation_predictions.csv"
predictions_df.to_csv(predictions_path, index=False)
print(f"\n💾 All predictions saved to {predictions_path}")

## 8. Prediction Analysis Visualization

In [ ]:
# Visualize prediction analysis
print("Creating prediction analysis visualization...")

fig = plot_prediction_analysis(
    predictions_df,
    top_n=50,
    figsize=(16, 12),
    save_path="../outputs/prediction_analysis.png"
)
plt.show()

print("Prediction analysis saved to ../outputs/prediction_analysis.png")

## 9. Embedding Space Visualization

In [ ]:
# Visualize learned embeddings in 2D
print("Creating embedding space visualization...")

# Get paper metadata for coloring
try:
    paper_metadata = extractor.get_paper_metadata()
    print(f"Loaded metadata for {len(paper_metadata)} papers")
except Exception as e:
    print(f"Could not load metadata: {e}")
    # Create dummy metadata
    paper_metadata = pd.DataFrame({
        'paper_id': list(entity_mapping.keys()),
        'citations': [0] * len(entity_mapping)
    })

# Get embeddings
with torch.no_grad():
    embeddings = trainer.model.entity_embeddings.weight.cpu()

# Sample embeddings for visualization (t-SNE is expensive)
sample_size = min(1000, len(embeddings))
sample_indices = torch.randperm(len(embeddings))[:sample_size]
sample_embeddings = embeddings[sample_indices]
sample_metadata = paper_metadata.iloc[sample_indices.numpy()] if len(paper_metadata) >= sample_size else paper_metadata

print(f"Visualizing {sample_size} embeddings with t-SNE...")

# Create embedding visualization
fig = plot_embedding_visualization(
    sample_embeddings,
    sample_metadata,
    method='tsne',
    figsize=(12, 8),
    save_path="../outputs/embedding_visualization.png"
)
plt.show()

print("Embedding visualization saved to ../outputs/embedding_visualization.png")

## 10. Model Confidence Analysis

In [ ]:
# Analyze model confidence and prediction quality
print("Analyzing model confidence and prediction patterns...")

# Score distribution analysis
score_stats = predictions_df['score'].describe()
print(f"\n📊 Prediction Score Statistics:")
print(f"• Mean: {score_stats['mean']:.4f}")
print(f"• Std: {score_stats['std']:.4f}")
print(f"• Min: {score_stats['min']:.4f}")
print(f"• Max: {score_stats['max']:.4f}")

# High confidence predictions (lowest scores)
high_confidence = predictions_df[predictions_df['score'] <= predictions_df['score'].quantile(0.1)]
print(f"\n🎯 High Confidence Predictions (top 10%):")
print(f"• Count: {len(high_confidence):,}")
print(f"• Score threshold: {predictions_df['score'].quantile(0.1):.4f}")
print(f"• These represent the most likely missing citations")

# Papers with most predictions
source_counts = predictions_df['source_paper'].value_counts().head(10)
print(f"\n📝 Papers with Most Predictions:")
for paper, count in source_counts.items():
    short_title = paper[:80] + "..." if len(paper) > 80 else paper
    print(f"• {count:2d} predictions: {short_title}")

# Papers most frequently predicted as targets
target_counts = predictions_df['target_paper'].value_counts().head(10)
print(f"\n🎯 Most Frequently Predicted Target Papers:")
for paper, count in target_counts.items():
    short_title = paper[:80] + "..." if len(paper) > 80 else paper
    print(f"• {count:2d} predictions: {short_title}")

## 11. Evaluation Summary and Insights

In [ ]:
# Comprehensive evaluation summary
print("\n" + "="*70)
print("📊 COMPREHENSIVE EVALUATION SUMMARY")
print("="*70)

print(f"\n🎯 Model Performance:")
print(f"   • Mean Reciprocal Rank: {results['mrr']:.4f} ({mrr_quality})")
print(f"   • Hits@1: {results['hits'][1]:.3f} ({results['hits'][1]*100:.1f}% top-1 accuracy)")
print(f"   • Hits@10: {results['hits'][10]:.3f} ({results['hits'][10]*100:.1f}% top-10 accuracy)")
print(f"   • AUC Score: {results['auc']:.4f} ({auc_quality})")

print(f"\n📋 Dataset Context:")
print(f"   • Test samples: {len(test_edges):,} ({len(test_pos_edges):,} positive, {len(test_neg_edges):,} negative)")
print(f"   • Entity vocabulary: {training_metadata['num_entities']:,} papers")
print(f"   • Network sparsity: {len(extractor.citation_edges) / (training_metadata['num_entities']**2):.6f}")

print(f"\n🔮 Prediction Generation:")
print(f"   • Generated predictions for {len(sample_papers)} papers")
print(f"   • Total predictions: {len(predictions_df):,}")
print(f"   • High-confidence predictions: {len(high_confidence):,}")
print(f"   • Score range: {predictions_df['score'].min():.4f} to {predictions_df['score'].max():.4f}")

print(f"\n💡 Key Insights:")
if results['hits'][1] > 0.1:
    print(f"   • Model successfully identifies missing citations with {results['hits'][1]*100:.1f}% top-1 accuracy")
else:
    print(f"   • Model struggles with precise ranking (low Hits@1)")

if results['auc'] > 0.8:
    print(f"   • Strong discrimination between citations and non-citations")
else:
    print(f"   • Moderate ability to distinguish citations from non-citations")

print(f"   • Average predicted rank for true citations: {1/results['mrr']:.1f}")
print(f"   • Model learned meaningful embeddings in {training_metadata['embedding_dim']}D space")

print(f"\n📁 Generated Files:")
print(f"   • ../outputs/evaluation_results.png - Performance metrics visualization")
print(f"   • ../outputs/prediction_analysis.png - Prediction analysis plots")
print(f"   • ../outputs/embedding_visualization.png - t-SNE embedding visualization")
print(f"   • ../outputs/citation_predictions.csv - All citation predictions")

print(f"\n🎓 Research Value:")
print(f"   • {len(high_confidence):,} high-confidence missing citations identified")
print(f"   • Demonstrates feasibility of ML-based citation recommendation")
print(f"   • Embeddings capture semantic relationships between papers")
print(f"   • Ready for notebook 04_story_visualization.ipynb")

print("\n✅ Prediction analysis complete! Model evaluation successful.")

In [ ]:
# Save evaluation results for story visualization
eval_summary = {
    'metrics': evaluation_results,
    'training_metadata': training_metadata,
    'prediction_stats': {
        'total_predictions': len(predictions_df),
        'high_confidence': len(high_confidence),
        'sample_papers': len(sample_papers),
        'score_range': (predictions_df['score'].min(), predictions_df['score'].max())
    },
    'interpretation': {
        'mrr_quality': mrr_quality,
        'auc_quality': auc_quality,
        'overall_assessment': overall
    }
}

with open('../outputs/evaluation_summary.pkl', 'wb') as f:
    pickle.dump(eval_summary, f)

print("Evaluation summary saved for story visualization.")

# Clean up
db.close()
if device.type == 'cuda':
    torch.cuda.empty_cache()

print("\nCleanup complete. Ready for story visualization!")